In [30]:
import notebook_common
import jaconv
from importlib import reload

reload(notebook_common)
from notebook_common import (
    # ------------------function------------------
    read_excel,
    to_csv_sig,
    # ------------------2400 word------------------
    r_word_2400_xlsx,
    w_word_2400_csv,
    w_word_2400_with_guide_csv,
    w_word_2400_without_guide_csv,
    # ------------------书面语相关------------------
    # 解析书面语正则
    pattern_table_written,
    pattern_td_written,
    r_written_freq_nlt_with_enter_txt,
    w_written_freq_nlt_with_enter_csv,
    w_written_freq_nlt_sumed_csv,
    # ------------------口语相关------------------
    # 解析口头语正则
    # pattern_table_spoken,
    # pattern_td_spoken,
    # r_spoken_freq_nlb_with_enter_txt,
    # w_spoken_freq_nlb_with_enter_csv,
    # w_spoken_freq_nlb_with_enter_filtered_csv,
    # w_spoken_freq_nlb_with_enter_filtered_hrt_csv,
    # w_spoken_freq_nlb_sumed_csv,
    # ------------------合并书面语和口语词频------------------
    w_merged_freq_sumed_csv,
    # ------------------合并书面语和口语词频------------------
    w_tmp_csv,
    # ------------------公共python库------------------
    pd,
    np,
    p,
    l,
)

p("Import common module")

Import common module


In [31]:
with open(r_written_freq_nlt_with_enter_txt, "r", encoding="utf-8") as f:
    lst = []
    content = f.read()  # 一次性读取
    table_matches = pattern_table_written.findall(content)  # 或 finditer 迭代匹配
    for table_match in table_matches:
        td_matches_lst = pattern_td_written.findall(table_match)
        lst.append(td_matches_lst)
    columns = ["headword", "reading", "roman", "freq", "freq_per", "rank"]
    df = pd.DataFrame(lst, columns=columns)
    to_csv_sig(df, w_written_freq_nlt_with_enter_csv)
    p("w_written_freq_nlt_with_enter_csv--ok")
df[0:3]

w_written_freq_nlt_with_enter_csv--ok


,headword,reading,roman,freq,freq_per,rank
0,110番,ヒャクトオバン,hyakutooban,108,0.0252,0.69889
1,110番する,ヒャクトオバンスル,hyakutoobansuru,7,0.0016,0.28913
2,1つ,ヒトツ,hitotsu,"4,366",1.0167,0.98121


In [32]:
# 指定列中，只要有一列的数据为空，删除该行,Inplace=True

# 如果headword中出现'-非自立'结尾，则删除
df['headword'] = df['headword'].str.replace(r'-非自立$', '', regex=True)

na_columns = ["headword", "reading", "freq", "rank"]
df.dropna(subset=na_columns, inplace=True)
# to_deleted_columns = ["rank", "freq_per", "roman"]
to_deleted_columns = ["rank", "freq_per"]
# 将无用的列删除
df.drop(to_deleted_columns, axis=1, inplace=True)
# 将词频列的字符串转化为数字
df["freq"] = pd.to_numeric(df["freq"].str.replace(",", ""), errors="coerce")
# 将发音reading转化为平假名
df["hira"] = df["reading"].apply(jaconv.kata2hira)
df[0:3]


,headword,reading,roman,freq,hira
0,110番,ヒャクトオバン,hyakutooban,108,ひゃくとおばん
1,110番する,ヒャクトオバンスル,hyakutoobansuru,7,ひゃくとおばんする
2,1つ,ヒトツ,hitotsu,4366,ひとつ


In [33]:
df["count-head-read"] = df.groupby(["headword", "reading"]).transform("size")
df["count-head-read-roman"] = df.groupby(["headword", "reading", "roman"]).transform(
    "size"
)
df[:3]

,headword,reading,roman,freq,hira,count-head-read,count-head-read-roman
0,110番,ヒャクトオバン,hyakutooban,108,ひゃくとおばん,1,1
1,110番する,ヒャクトオバンスル,hyakutoobansuru,7,ひゃくとおばんする,1,1
2,1つ,ヒトツ,hitotsu,4366,ひとつ,1,1


In [34]:
# 以拼写,发音,罗马音来分组，对词频进行汇总
summed_df = (
    df["freq"].groupby([df["headword"], df["hira"], df["roman"]]).sum().to_frame()
)
summed_df.rename(columns={"freq": "written_freq"}, inplace=True)
summed_df.sort_values(by="written_freq", ascending=False, inplace=True)

summed_df = summed_df[
    ~(
        (summed_df.index.get_level_values("headword") == "する")
        & (summed_df["written_freq"] < 300)
    )
]
to_csv_sig(summed_df, w_written_freq_nlt_sumed_csv, need_index=True)
p("w_written_freq_nlt_sumed_csv--ok")
summed_df

w_written_freq_nlt_sumed_csv--ok


,,,written_freq
headword,hira,roman,
いる,いる,iru,1067608
こと,こと,koto,701470
の,の,no,686025
する,する,suru,607405
なる,なる,naru,528539
...,...,...,...
焼きいもする,やきいもする,yakiimosuru,1
へろへろする,へろへろする,heroherosuru,1
へなへなする,へなへなする,henahenasuru,1
